In [1]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [2]:
torch.manual_seed(0)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .1 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 2000
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [3]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [5]:
def calc_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [6]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) 
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            if iter <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #l2_norm.backward()
                #norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                neg_ELBO = -log_lik.mean() - obs_model(C_PATH) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\n C_PATH =', C_PATH)
                neg_ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [7]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022811.225. Best norm loss value is: 228112.25.

C_PATH mean = tensor([[0.7400, 0.6597, 0.7890],
        [0.7433, 0.6603, 0.7777]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.7279,  0.9477,  0.6554],
         [ 0.7301,  0.5975,  0.9460],
         ...,
         [ 0.7950,  0.6977,  0.8644],
         [ 0.7731,  0.7774,  0.4001],
         [ 0.6925,  0.5654,  0.0750]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.7279,  0.6799,  1.0443],
         [ 0.9856,  0.2549,  0.5817],
         ...,
         [ 0.7855,  0.6523,  0.5108],
         [ 0.5117,  0.5873,  0.4136],
         [ 1.1247,  0.7128,  0.1988]]], grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [00:30<00:02,  2.58s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000008e+19. Best ELBO loss value is: 109122.8125.

C_PATH mean = tensor([[2.9936, 0.2928, 0.9053],
        [3.0691, 0.2964, 0.9061]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.2225,  0.3699,  0.5858],
         [ 3.0752,  0.1051,  0.7360],
         ...,
         [ 0.6558,  0.3566,  1.2980],
         [ 1.6749,  0.3363,  0.8938],
         [ 1.3371,  0.4383,  0.6796]],

        [[45.6603,  0.0715,  0.7147],
         [ 2.2029,  0.5779,  0.3308],
         [ 0.1716,  0.6693,  0.2791],
         ...,
         [ 2.7447,  0.5886,  1.1566],
         [ 0.5959,  0.7045,  1.1216],
         [ 0.5750,  0.5827,  0.5787]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 11/11 [00:32<00:00,  2.95s/it]


In [8]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000022654.498438. Best norm loss value is: 226544.984375.

C_PATH mean = tensor([[0.9213, 0.6939, 0.6697],
        [0.9215, 0.6938, 0.6714]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 0.8887,  0.9192,  0.6469],
         [ 0.8367,  0.7065,  0.6372],
         ...,
         [ 0.7734,  0.6834,  0.5489],
         [ 0.7954,  0.6465,  0.5245],
         [ 0.7759,  0.5440,  0.5987]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6497,  0.9146,  0.6413],
         [ 0.8030,  0.8280,  0.6850],
         ...,
         [ 0.7383,  0.6963,  0.6824],
         [ 0.8768,  0.7061,  0.4931],
         [ 0.8344,  0.6408,  0.7958]]], grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [00:24<00:02,  2.29s/it]

Moving average ELBO loss at 10 iterations is: 5.0000000000000156e+19. Best ELBO loss value is: 291740.25.

C_PATH mean = tensor([[3.4132, 0.3701, 0.4326],
        [3.3868, 0.3704, 0.4334]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[45.6603,  0.0715,  0.7147],
         [ 1.4470,  0.7786,  0.3114],
         [ 0.9272,  2.4852,  0.5729],
         ...,
         [ 1.5159,  0.3224,  0.4725],
         [ 4.5030,  0.3151,  0.4817],
         [ 3.0679,  0.2824,  0.8820]],

        [[45.6603,  0.0715,  0.7147],
         [ 0.6071,  3.0910,  0.5213],
         [ 3.4074,  0.9432,  0.2681],
         ...,
         [ 5.4625,  0.7837,  0.2615],
         [ 1.2688,  0.3830,  0.3958],
         [ 1.3389,  0.3429,  0.6935]]], grad_fn=<CatBackward>)



Train Diffusion: 100%|██████████| 11/11 [00:26<00:00,  2.38s/it]


In [9]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000026889.896875. Best norm loss value is: 268898.96875.

C_PATH mean = tensor([[0.8756, 0.6013, 0.6247, 0.6046],
        [0.8197, 0.6025, 0.6287, 0.6043]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [9.9940e-01, 1.5280e+00, 5.9410e-01, 5.7048e-01],
         [8.3784e-01, 1.1969e+00, 5.4350e-01, 6.7734e-01],
         ...,
         [3.5875e-01, 6.9492e-01, 6.5304e-01, 6.0681e-01],
         [6.8057e-01, 6.6072e-01, 6.7103e-01, 6.6171e-01],
         [9.9727e-01, 6.0029e-01, 6.8502e-01, 5.3131e-01]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [7.4333e-01, 2.8711e-01, 6.8474e-01, 6.9089e-01],
         [4.0082e-01, 4.1919e-01, 7.5749e-01, 5.2553e-01],
         ...,
         [1.1247e+00, 4.4564e-01, 5.7925e-01, 5.5629e-01],
         [7.0672e-01, 6.1570e-01, 6.4945e-01, 5.0325e-01],
         [3.9055e-01, 6.3115e-01, 5.3313e-01, 6.6306e-01]]],
       grad_fn=<CatBac


Train Diffusion:  91%|█████████ | 10/11 [00:36<00:04,  4.13s/it]

Moving average ELBO loss at 10 iterations is: 5.00000000000017e+19. Best ELBO loss value is: 999448.8125.

C_PATH mean = tensor([[3.6303, 0.2625, 0.3114, 0.5251],
        [3.5973, 0.2629, 0.3133, 0.5700]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [6.1405e-01, 6.1618e-01, 3.3340e-01, 1.8146e-01],
         [5.7157e-01, 3.6089e-01, 2.6173e-01, 5.2516e-01],
         ...,
         [4.4637e+00, 2.4523e-01, 2.9430e-01, 1.3415e-01],
         [1.0603e+00, 3.0313e-01, 2.5601e-01, 3.4446e+00],
         [4.8469e-01, 2.5845e-01, 6.3324e-01, 2.5314e-01]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [5.7834e-01, 2.0412e+00, 2.5363e-01, 4.4484e-01],
         [1.5397e+00, 8.7900e-01, 3.3122e-01, 1.9905e-01],
         ...,
         [5.5114e-01, 3.0995e-01, 2.6297e-01, 2.0336e+00],
         [3.5203e+00, 2.2567e-01, 3.2567e-01, 4.4911e-02],
         [4.2851e+00, 3.2481e-01, 2.3575e-01, 2.0464e+00]]],
       grad_fn=<Ca


Train Diffusion: 100%|██████████| 11/11 [00:41<00:00,  3.74s/it]


In [10]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000027061.1. Best norm loss value is: 270611.0.

C_PATH mean = tensor([[0.7143, 0.6234, 0.7119, 0.7436],
        [0.7186, 0.6246, 0.7109, 0.7385]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [3.6299e-01, 7.8679e-01, 6.4215e-01, 7.6002e-01],
         [7.4314e-01, 5.0249e-01, 8.4127e-01, 1.3014e+00],
         ...,
         [6.1262e-01, 5.4987e-01, 4.6918e-01, 2.1939e-01],
         [6.7402e-01, 5.6650e-01, 8.2729e-01, 2.1840e-02],
         [8.1518e-01, 7.7214e-01, 9.8227e-01, 9.2543e+00]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [5.0743e-01, 9.3998e-01, 5.4507e-01, 8.6198e-01],
         [7.3106e-01, 7.0964e-01, 5.2576e-01, 6.0665e-01],
         ...,
         [7.6069e-01, 6.3580e-01, 1.0080e+00, 1.9280e+00],
         [6.7824e-01, 8.0711e-01, 7.6677e-01, 1.8866e+00],
         [7.6165e-01, 6.9299e-01, 3.4567e-01, 5.8458e-03]]],
       grad_fn=<CatBackward>)



Train Diffusion:  91%|█████████ | 10/11 [01:07<00:06,  6.81s/it]

Moving average ELBO loss at 10 iterations is: 5.0000000000000565e+19. Best ELBO loss value is: 943401.9375.

C_PATH mean = tensor([[2.9507, 0.5804, 0.3800, 0.2141],
        [3.0321, 0.5606, 0.3866, 0.2122]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [3.6704e-01, 2.2736e+00, 5.2126e-01, 2.3374e-01],
         [1.4583e+00, 1.3037e+00, 4.0033e-01, 2.5541e-01],
         ...,
         [1.8846e+00, 3.9993e-01, 2.4275e-01, 8.2365e-02],
         [2.8080e+00, 4.1401e-01, 2.8277e-01, 2.1487e-01],
         [1.0457e+00, 5.2991e-01, 2.2198e-01, 9.0876e-02]],

        [[5.3602e+01, 1.1421e-01, 6.7073e-01, 1.3415e-02],
         [1.2566e+00, 8.5686e-01, 4.0615e-01, 3.6185e-01],
         [3.3945e+00, 3.4486e-01, 5.8059e-01, 3.5940e-01],
         ...,
         [3.2020e+00, 5.0570e-01, 3.2429e-01, 2.5977e-01],
         [1.2080e+00, 6.1714e-01, 3.0447e-01, 9.2791e-02],
         [2.7005e+00, 4.0357e-01, 3.2321e-01, 5.6443e-01]]],
       grad_fn=<


Train Diffusion: 100%|██████████| 11/11 [01:10<00:00,  6.44s/it]


In [11]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000027074.721874. Best norm loss value is: 270747.21875.

C_PATH mean = tensor([[0.7535, 0.7924, 0.6640, 0.7536],
        [0.7470, 0.7780, 0.6686, 0.7574]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.1703e+00, 5.3442e-01, 7.3103e-01, 6.4130e-01],
         [1.5373e-01, 7.6910e-01, 6.4898e-01, 7.1473e-01],
         ...,
         [5.9314e-01, 8.9921e-01, 6.0788e-01, 5.6136e-01],
         [7.1941e-01, 6.8893e-01, 6.8239e-01, 6.4098e-01],
         [6.9337e-01, 6.7578e-01, 1.0404e+00, 4.9995e-01]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [9.3223e-01, 1.6799e+00, 4.8578e-01, 5.9194e-01],
         [7.9274e-01, 4.3457e-01, 7.6946e-01, 7.3061e-01],
         ...,
         [6.0268e-01, 6.9652e-01, 7.0334e-01, 8.3120e-01],
         [6.0113e-01, 5.9995e-01, 7.9828e-01, 7.9510e-01],
         [4.5336e-01, 5.8754e-01, 7.4844e-01, 7.4450e-01]]],
       grad_fn=<CatBac


Train Diffusion:  91%|█████████ | 10/11 [00:57<00:06,  6.18s/it]

Moving average ELBO loss at 10 iterations is: 5.000000000000093e+19. Best ELBO loss value is: 939787.0625.

C_PATH mean = tensor([[2.7449, 0.7305, 0.4820, 0.3269],
        [2.7274, 0.7302, 0.4809, 0.3242]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.2117e+00, 7.5191e-01, 7.7708e-01, 7.0159e-01],
         [2.6366e+00, 4.3881e+00, 5.1664e-01, 1.9567e-01],
         ...,
         [1.0886e+00, 7.2616e-01, 1.8250e-01, 6.1034e-01],
         [5.4818e+00, 4.9457e-01, 1.8401e-01, 6.8846e-02],
         [3.7332e-01, 7.7025e-01, 5.4459e-01, 6.6613e-02]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.0866e+00, 5.6398e+00, 4.9667e-01, 1.5482e-01],
         [5.8435e-01, 8.8152e-01, 7.8000e-01, 4.2756e-01],
         ...,
         [4.6123e+00, 4.3976e-01, 5.2204e-01, 4.1713e-02],
         [8.9684e-01, 9.1660e-01, 8.3135e-01, 8.4085e-01],
         [4.9204e+00, 4.6775e-01, 2.0005e-01, 7.6381e-01]]],
       grad_fn=<C


Train Diffusion: 100%|██████████| 11/11 [01:02<00:00,  5.72s/it]


In [12]:
train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA)


Train Diffusion:   0%|          | 0/11 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000026829.665625. Best norm loss value is: 268296.65625.

C_PATH mean = tensor([[0.9391, 0.7313, 0.6611, 0.5455],
        [0.9432, 0.7312, 0.6611, 0.5452]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [7.6318e-01, 9.8736e-01, 7.3540e-01, 6.4607e-01],
         [6.8033e-01, 8.2627e-01, 7.6372e-01, 6.6260e-01],
         ...,
         [9.9666e-01, 6.4746e-01, 6.6859e-01, 4.6739e-01],
         [1.1023e+00, 7.6750e-01, 5.4918e-01, 2.7294e-01],
         [9.7383e-01, 6.8512e-01, 6.3329e-01, 1.2391e+00]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [7.6318e-01, 7.7522e-01, 9.4605e-01, 6.3471e-01],
         [8.8614e-01, 7.9865e-01, 7.1846e-01, 5.6408e-01],
         ...,
         [8.5151e-01, 8.5384e-01, 5.3274e-01, 4.6862e-01],
         [9.0025e-01, 5.9583e-01, 6.3099e-01, 2.5325e-01],
         [5.9562e-01, 8.9964e-01, 6.4710e-01, 1.5409e+00]]],
       grad_fn=<CatBac


Train Diffusion:  91%|█████████ | 10/11 [00:40<00:04,  4.11s/it]

Moving average ELBO loss at 10 iterations is: 5.00000000000011e+19. Best ELBO loss value is: 1164486.625.

C_PATH mean = tensor([[2.1742, 0.5104, 0.5643, 0.4610],
        [2.1810, 0.4985, 0.5587, 0.4610]], grad_fn=<MeanBackward1>)

 C_PATH = tensor([[[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [3.6165e-01, 9.6191e-01, 3.2046e-01, 5.5201e-01],
         [6.8956e-01, 6.7154e-01, 3.5985e-01, 6.7055e-01],
         ...,
         [1.6042e+00, 6.8118e-01, 2.9164e-01, 2.1181e-01],
         [1.4482e+00, 8.7103e-01, 5.8514e-01, 8.2903e-02],
         [1.7209e+00, 8.7628e-01, 3.2675e-01, 6.4895e-01]],

        [[5.3606e+01, 1.9081e-01, 6.7073e-01, 1.3415e-02],
         [1.6794e+00, 1.5872e+00, 5.1270e-01, 3.0763e-01],
         [1.6210e+00, 1.5457e+00, 5.9144e-01, 2.6248e-01],
         ...,
         [2.0975e+00, 2.5883e-01, 5.6856e-01, 6.0182e-01],
         [2.0876e+00, 3.5834e-01, 2.7190e-01, 1.1087e+00],
         [1.3318e+00, 4.2882e-01, 6.0033e-01, 9.0724e-01]]],
       grad_fn=<Ca


Train Diffusion: 100%|██████████| 11/11 [00:43<00:00,  3.99s/it]
